In [2]:
from transformer_lens import HookedTransformer
from mechinterp.Interpreter import Interpreter

In [3]:
model = HookedTransformer.from_pretrained("EleutherAI/pythia-12b-deduped")
interp = Interpreter(model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/morg/students/maorlavi/miniconda3/envs/probing_fakepedia/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model EleutherAI/pythia-12b-deduped into HookedTransformer


In [4]:
logits, cache = model.run_with_cache("It was the best of times")

In [7]:
interp.tuned_lens(cache["blocks.15.hook_resid_post"][0, 5], l=15, k=4)


Tuned Logit Lens Output:
	- Topk tokens: [' times', ' all', '\n', ' and']

	- Bottomk tokens: ['theless', 'medsc', '        ', '     ']

In [36]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tuned_lens import TunedLens

model_name = "EleutherAI/pythia-12b-deduped"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

lens = TunedLens.from_model(model)
input_text = "It was the best of times"
inputs = tokenizer(input_text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)
    hidden_states = outputs.hidden_states  
l = 15
k = 10
hidden_state_l = hidden_states[l]
logits = lens(h = hidden_state_l, idx = l)

top_k_probs, top_k_indices = torch.topk(logits[0, 3, :], k)

top_k_tokens = [tokenizer.decode([idx]) for idx in top_k_indices]

print(top_k_tokens)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[' possible', ' thing', '-', ' way', ' part', ' place', ' I', ' kind', ' we', ' time']


In [50]:
from tuned_lens import load_artifacts

artifacts = load_artifacts.load_lens_artifacts("EleutherAI/pythia-12b-deduped")

In [47]:
artifacts[1]

PosixPath('/home/morg/students/maorlavi/models/huggingface/hub/spaces--AlignmentResearch--tuned-lens/snapshots/1ac7285852a22309f571c2555efc37375d0c4cda/lens/EleutherAI/pythia-12b-deduped/params.pt')

In [48]:
l = 23
k = 10
hidden_state_l = hidden_states[l+2]
logits = lens(h = hidden_state_l, idx = l)

top_k_probs, top_k_indices = torch.topk(logits[0, 4, :], k)

top_k_tokens = [tokenizer.decode([idx]) for idx in top_k_indices]

print(top_k_tokens)

[' times', ' all', '\n', ' life', ' the', ' time', 's', ' T', ' and', '-']


In [19]:
print(len(hidden_states))

37
